In [5]:
from cobra.io import read_sbml_model
import model_methods
from model_methods import get_base_model, update_medium, add_sucrose
import warnings

warnings.filterwarnings("ignore", ".*not in.*", UserWarning)

model = get_base_model()
model_old = read_sbml_model('iNF517.xml')

biomass_obj = model_old.objective
objective = model.objective

In [2]:
print(biomass_obj)

Maximize
1.0*BIOMASS_LLA - 1.0*BIOMASS_LLA_reverse_0796e


## Exploring the original validated model.

In [3]:
from cobra.flux_analysis import flux_variability_analysis

with model:
    reactions_of_interest = [
        model.reactions.DM_mqn7_c,
        model.reactions.BIOMASS_LLA,  # Biomass reaction
        model.reactions.EX_glc__D_e, # Glucose uptake reaction
        model.reactions.EX_o2_e,    # Oxygen uptake reaction
        model.reactions.ATPM,        # ATP maintenance reaction
        model.reactions.EX_sucr_e,     # Sucrose uptake reaction (alternative)
        model.reactions.G3PD1ir,        # NADH producing reaction
        model.reactions.EX_glu__L_e
    ]

    fva = flux_variability_analysis(
        model,
        reaction_list=reactions_of_interest,
        fraction_of_optimum=0.9
    )
    
print(fva, "\n")

                  minimum      maximum
DM_mqn7_c    3.600000e-02     0.040000
BIOMASS_LLA  1.102429e-17     0.004263
EX_glc__D_e -2.120000e+00    -1.040863
EX_o2_e      0.000000e+00     0.000000
ATPM         0.000000e+00     4.138183
EX_sucr_e    0.000000e+00     0.000000
G3PD1ir     -9.997636e+02  1000.000000
EX_glu__L_e -5.000000e-02    -0.046000 



The Flux Variability Analysis (FVA) results for MK-7 production in the *L. lactis* model indicate that the optimized flux through the DM_mqn7_c reaction ranges from 0.036 to 0.04 mmol/gDW/h. 
* The model **prioritizes** MK-7 production over biomass synthesis (BIOMASS_LLA), with a maximum biomass flux of approximately 0.00426 mmol/gDW/h. 
* **Glucose uptake** (EX_glc__D_e) is allowed within the range of -2.12 to -1.04 mmol/gDW/h, indicating the flexibility of the model in utilizing glucose as a carbon source. 
* Oxygen uptake (EX_o2_e) is predicted to be zero, suggesting **anaerobic** conditions. 
* The **ATP maintenance** flux (ATPM) varies from 0.0 to 4.14 mmol/gDW/h, reflecting the energy demands associated with MK-7 synthesis. 
* Sucrose uptake (EX_sucr_e) is predicted to be zero, indicating no contribution from these reactions to MK-7 production. 
* G3PD1ir has a big range of 2000. The negative flux values suggest the potential for consumption of substrates (dhap_c and nadh_c), and the positive flux values indicate the potential for the production of products (glyc3p_c and nad_c).

## Exploring the cell with updated medium.

In [6]:
model = add_sucrose(update_medium(model))
model.medium

{'EX_4abz_e': 0.00999,
 'EX_ade_e': 0.01014,
 'EX_ala__L_e': 0.04,
 'EX_arg__L_e': 1,
 'EX_asp__L_e': 0.01,
 'EX_co2_e': 1000.0,
 'EX_cys__L_e': 0.02,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 2.12,
 'EX_glu__L_e': 16.05,
 'EX_gly_e': 0.04,
 'EX_gua_e': 0.00906,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_his__L_e': 0.01,
 'EX_ile__L_e': 0.05,
 'EX_ins_e': 0.00255,
 'EX_leu__L_e': 0.06,
 'EX_lys__L_e': 0.04,
 'EX_met__L_e': 0.01,
 'EX_mn2_e': 1000.0,
 'EX_nac_e': 0.00111,
 'EX_nh4_e': 0.59,
 'EX_orot_e': 0.00439,
 'EX_phe__L_e': 0.02,
 'EX_pi_e': 1000.0,
 'EX_pnto__R_e': 0.00062,
 'EX_ribflv_e': 0.00036,
 'EX_ser__L_e': 0.11,
 'EX_sucr_e': 284,
 'EX_thm_e': 0.00041,
 'EX_thr__L_e': 0.06,
 'EX_thymd_e': 0.00283,
 'EX_ura_e': 0.01222,
 'EX_val__L_e': 0.05,
 'EX_xan_e': 0.00901,
 'EX_zn2_e': 1000.0}

In [7]:
with model:
    reactions_of_interest = [
        model.reactions.DM_mqn7_c,
        model.reactions.BIOMASS_LLA,  
        model.reactions.EX_glc__D_e,
        model.reactions.EX_o2_e,               
        model.reactions.ATPM,              
        model.reactions.EX_sucr_e,           
        model.reactions.G3PD1ir,
        model.reactions.EX_glu__L_e 
    ]

    fva = flux_variability_analysis(
        model,
        reaction_list=reactions_of_interest,
        fraction_of_optimum=0.9
    )
    
print(fva, "\n")

                minimum      maximum
DM_mqn7_c      9.988506    11.098340
BIOMASS_LLA    0.000000     0.103424
EX_glc__D_e   -2.120000    -0.920000
EX_o2_e        0.000000     0.000000
ATPM           0.000000    76.208605
EX_sucr_e   -284.000000  -254.685931
G3PD1ir     -349.131089  1000.000000
EX_glu__L_e  -16.050000    -9.988506 



<font size='3'>  
    
The optimized flux through DM_mqn7_c has increased, ranging from 9.99 to 11.10 mmol/gDW/h, indicating a substantial **enhancement in MK-7 production.** 

* Biomass synthesis (BIOMASS_LLA) now ranges from 0.0 to 0.1034 mmol/gDW/h, suggesting a potential trade-off between biomass and Mk-7 production. Glutamate and sucrose addition **improved** it as well. 
* **Glucose uptake** (EX_glc__D_e) remains within the range of -2.12 to -0.92 mmol/gDW/h, indicating continued utilization of glucose in **slightly lower** amounts. 
* Oxygen uptake (EX_o2_e) remains at zero, implying the persistence of **anaerobic** conditions. 
* The ATP maintenance flux (ATPM) has notably increased, ranging from 0.0 to 76.21 mmol/gDW/h, reflecting the **higher energy demands** associated with elevated MK-7 production. 
* **Sucrose** uptake (EX_sucr_e) has been introduced at a substantial rate, ranging from -284.0 to -254.69 mmol/gDW/h. So, the cell is using it as a carbon-source.
* Additionally, **glutamate** uptake (EX_glu__L_e) has been introduced at a rate ranging from -16.05 to -10.0 mmol/gDW/h. 
* There is a big change towards a positive size in the negative flux of G3PD1ir, meaning that model now allows a **lower rate of consumption of substrates (NADH).** NADH is a critical cofactor in cellular functioning, therefore the change in the negative boundary indicates a better cell functioning with newly introduced molecules, as it requires less cofactor.
    
</font>

## Adding ATP into the medium

An increase in ATP consumption suggests an influence of ATP on the production. There is no exchange reaction between cytosol and external environment for ATP, so it was decided to add it to try to increase the production further.

In [8]:
from cobra import Metabolite, Reaction
# addition of an exchange reaction for ATP between external and internal comparm

model.remove_reactions(['EX_atp_e'])
exchange_reaction = Reaction('EX_atp_e') 
atp_e = Metabolite(id='atp_e', compartment='e', 
                   name='ATP_external')

exchange_reaction.add_metabolites({model.metabolites.get_by_id('atp_c'): 1,
                                   atp_e: -1
                                  })

model.add_reactions([exchange_reaction])

In [9]:
# restricting the reverse reaction: atp_c -> atp_e
atp_exchange = model.reactions.get_by_id("EX_atp_e")
atp_exchange.lower_bound = -100
model.reactions.EX_atp_e

Reaction identifier,EX_atp_e
Name,
Memory address,0x1625320d0
Stoichiometry,atp_e <=> atp_c ATP_external <=> ATP C10H12N5O13P3
GPR,
Lower bound,-100
Upper bound,1000.0


In [10]:
# adding ATP to the medium
with model:
    original_medium = model.medium
    original_medium['EX_atp_e'] = 100
    model.medium = original_medium
    print(model.optimize().objective_value)

EX_atp_e does not seem to be an an exchange reaction. Applying bounds anyway.


11.098340473300974


Addition of ATP to the medium did not improve the production.

## Adding oxygen. 

Now, the model fully runs in an anaerobic mode. *L. lactis* is a facultative anaerobe, so it could function in the presence of O2 as well. An attempt to change the productivity based on that:

In [11]:
# O2t: o2_e --> o2_c, O2 transport  diffusion
o2_exchange = model.reactions.get_by_id("O2t")
o2_exchange.upper_bound = 100

with model:
    original_medium['EX_o2_e'] = 1000
    model.medium = original_medium
    print(model.optimize().objective_value)

EX_atp_e does not seem to be an an exchange reaction. Applying bounds anyway.


11.098340473300988


Addition of O2 also did not change the outcome.

## Assessing the pathways targeted in a research paper

### A model with sucrose

In [12]:
with model:
    reactions_of_interest = [model.reactions.MEVK1,
                             model.reactions.DHNAOT7,
                             model.reactions.MQNS,
                             model.reactions.PREN, 
                             model.reactions.DM_mqn7_c]
    
    fva = flux_variability_analysis(model, 
                                    reaction_list = reactions_of_interest, 
                                    fraction_of_optimum=0.9)

print(fva, "\n")

             minimum    maximum
MEVK1      69.919545  77.688383
DHNAOT7     9.988506  11.098340
MQNS        0.000000  11.098340
PREN        9.988506  11.098340
DM_mqn7_c   9.988506  11.098340 



**MEVK1**:
The flux through the MEVK1 reaction can vary between approximately 69.92 and 77.69. This means that the model allows for flexibility in the utilization of mevalonate kinase in the mevalonate pathway, and this range provides insights into the potential variability of this reaction under different conditions.

It is interesting to note that the fluxes through **DHNAOT7, MQNS** and **PREN** have the same maximum value as DM_mqn7_c, and the minimum values for DHNAOT7 and PREN are also the same. This could suggest a direct relationship between these fluxes and the production of mqn7, especially for DHNAOT7 and PREN.

### A model without sucrose

As can be seen, an extreme sucrose addition is not affecting the correlation of these pathways with mqn7 production. The boundaries are still matching when getting the model back to the "low sucrose" medium.

In [13]:
model = update_medium(get_base_model())

with model:
    reactions_of_interest = [model.reactions.MEVK1,
                             model.reactions.DHNAOT7,
                             model.reactions.MQNS,
                             model.reactions.PREN, 
                             model.reactions.DM_mqn7_c]
    
    fva = flux_variability_analysis(model, 
                                    reaction_list = reactions_of_interest, 
                                    fraction_of_optimum=0.9)

print(fva, "\n")

            minimum   maximum
MEVK1      0.939571  1.043967
DHNAOT7    0.134224  0.149138
MQNS       0.000000  0.149138
PREN       0.134224  0.149138
DM_mqn7_c  0.134224  0.149138 



## Analysis of the boundaries

It can be seen that all the fluxes are already at their maximum capacity. When the lower boundaries for each of them were adjusted to higher values, the status was infeasible, so the model is not allowing to further increase the production by increasing the fluxes rates. The reason might be, for example, an insufficient amount of metabolites or substrate. 
Nevertheless, with the FVA it was proved that these pathways play a role in mqn7 production, so there is a potential to facilitate an increase in these fluxes to obtain more vitamin K.

In [29]:
solution = model.optimize()
print(f"Fluxes values:\n "
      f"MEVK1: {solution.fluxes.MEVK1},\n "
      f"DHNAOT7: {solution.fluxes.DDPA},\n "
      f"MQNS: {solution.fluxes.MQNS},\n "
      f"PREN: {solution.fluxes.PREN}")

Fluxes values:
 MEVK1: 77.68838331310691,
 DHNAOT7: 11.098340473300983,
 MQNS: 11.098340473300984,
 PREN: 11.098340473300983


In [24]:
with model:
    reaction_name = "MQNS"
    reaction = model.reactions.get_by_id(reaction_name)
    new_lower_bound = 11.1
    reaction.lower_bound = new_lower_bound
    print(model.optimize().objective_value)

11.098340473300984


/Users/kseniakirdey/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
